In [1]:
import os
import cv2
import itertools
import librosa
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import soundfile as sf
from pydub import AudioSegment
from tqdm import tqdm
from glob import glob
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Pytorch
import torch
import torchaudio
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchaudio.transforms import MelSpectrogram, MFCC, SpectralCentroid

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Pacotes para o relatório de hardware
import gc
import types
import pkg_resources
import pytorch_lightning as pl

# Seed para reproduzir os mesmos resultados
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

device = 'cpu'

c:\Users\victor.serra\AppData\Local\Programs\Python\Python39\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


### Import dos dados e cálculo do desvio

In [2]:
audio_folder = 'chords/variation_chord_audio/'

In [3]:
labels = pd.read_csv('labels.csv')
labels['file_path'] = labels['file_name'].apply(lambda x: os.path.join(audio_folder, x))
labels['chord_idx'] = pd.Categorical(labels['chord']).codes

In [4]:
transforms_dict = {
    'mel_spectrogram': torchaudio.transforms.MelSpectrogram(sample_rate=22050),
    'mfcc': torchaudio.transforms.MFCC(sample_rate=22050),
    'spectral': torchaudio.transforms.SpectralCentroid(sample_rate=22050),
    'chroma': lambda x: torch.tensor(librosa.feature.chroma_stft(y=x.numpy(), sr=22050)),
    'tonnetz': lambda x: torch.tensor(librosa.feature.tonnetz(y=x.numpy(), sr=22050)),
}

In [5]:
def calculate_feature_statistics(audio_paths, feature_type):
    assert feature_type in transforms_dict, f"Feature {feature_type} is not supported."
    
    # Obtenha a transformação específica
    transform = transforms_dict[feature_type]
    
    # Inicialize acumuladores
    sum_of_features = None
    sum_of_features_squared = None
    num_features = 0
    
    # Loop sobre os caminhos de áudio
    for path in tqdm(audio_paths):
        # Carrega o áudio
        waveform, sample_rate = torchaudio.load(path)
        # Converte para o domínio da frequência e aplica a transformação
        if callable(transform):
            feature = transform(waveform.squeeze(0))  # assumindo que waveform é um tensor 2D [channels, time]
        else:
            feature = transform(waveform)
        
        # Converte para tensor se ainda não for um
        if not isinstance(feature, torch.Tensor):
            feature = torch.from_numpy(np.array(feature))
        
        feature_mean = feature.mean(dim=-1)
        feature_squared_mean = (feature ** 2).mean(dim=-1)
        
        # Inicializa os acumuladores se ainda não estiverem inicializados
        if sum_of_features is None:
            sum_of_features = torch.zeros_like(feature_mean)
            sum_of_features_squared = torch.zeros_like(feature_mean)
        
        # Atualiza os acumuladores
        sum_of_features += feature_mean
        sum_of_features_squared += feature_squared_mean
        num_features += 1
    
    # Calcula média e desvio padrão
    mean = sum_of_features / num_features
    std = (sum_of_features_squared / num_features - mean ** 2).sqrt()
    
    return mean, std

In [6]:
normMean, normStd = calculate_feature_statistics(labels['file_path'], 'mel_spectrogram')

100%|██████████| 37131/37131 [09:01<00:00, 68.55it/s] 


### Funções do modelo

In [7]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
def inicializa_modelo(num_classes, feature_extract, audio_feature_type='mel_spectrogram', use_pretrained=True):
    input_size = 224  # Este é o tamanho padrão para o modelo Densenet

    # Inicializa o número de canais de entrada com base na característica de áudio selecionada
    feature_channels = {
        'mel_spectrogram': 1,
        'mfcc': 13,
        'chroma': 12,
        'spectral_centroid': 1,  
        'tonnetz': 6
    }

    num_input_channels = feature_channels.get(audio_feature_type, 1)  # Padrão para espectrograma de mel

    model_ft = models.densenet121(pretrained=use_pretrained)
    
    set_parameter_requires_grad(model_ft, feature_extract)

    model_ft.features.conv0 = nn.Conv2d(num_input_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    num_ftrs = model_ft.classifier.in_features
    
    model_ft.classifier = nn.Linear(num_ftrs, num_classes)
    return model_ft, input_size

In [9]:
class AudioDataset(Dataset):
    def __init__(self, df, audio_feature_type='mel_spectrogram', transform=None):
        self.df = df
        self.audio_feature_type = audio_feature_type
        self.transform = transform

    def __getitem__(self, index):
        audio_path = self.df['file_path'][index]
        label = torch.tensor(int(self.df['chord_idx'][index]))

        if self.audio_feature_type in ['mel_spectrogram', 'mfcc']:
            waveform, sample_rate = torchaudio.load(audio_path)

            if self.audio_feature_type == 'mel_spectrogram':
                feature = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate)(waveform)
            elif self.audio_feature_type == 'mfcc':
                feature = torchaudio.transforms.MFCC(sample_rate=sample_rate)(waveform)
        elif self.audio_feature_type in ['chroma', 'tonnetz']:
            y, sr = librosa.load(audio_path)
            
            if self.audio_feature_type == 'chroma':
                feature = librosa.feature.chroma_stft(y=y, sr=sr)
            elif self.audio_feature_type == 'tonnetz':
                feature = librosa.feature.tonnetz(y=y, sr=sr)

            # Converte a característica para um tensor PyTorch
            feature = torch.tensor(feature).float()
        else:
            raise ValueError(f"Feature type {self.audio_feature_type} not recognized.")

        if self.transform:
            feature = self.transform(feature)

        return feature, label

    def __len__(self):
        return len(self.df)

In [10]:
# Função para calcular erro em treino e validação durante o treinamento
class CalculaMetricas(object):
    
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [11]:
# Listas para erro e acurácia em treino
total_loss_train, total_acc_train = [],[]
# Função de treino do modelo
def treina_modelo(treino_loader, model, criterion, optimizer, epoch):
    
    # Coloca o modelo em modo de treino
    model.train()
    
    # Inicializa objetos de cálculo de métricas
    train_loss = CalculaMetricas()
    train_acc = CalculaMetricas()
    
    # Iteração
    curr_iter = (epoch - 1) * len(treino_loader)
    
    # Loop de treino
    for i, data in enumerate(treino_loader):
        
        # Extra os dados
        images, labels = data
        
        # Tamanho da imagem
        N = images.size(0)
        
        # Coloca imagens e labels no device
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        # Zera os gradientes
        optimizer.zero_grad()
        
        # Previsão do modelo
        outputs = model(images)

        # Erro do modelo
        loss = criterion(outputs, labels)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        # Obtem a previsão de maior probabilidade
        prediction = outputs.max(1, keepdim = True)[1]
        
        # Atualiza as métricas
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        
        # Iteração
        curr_iter += 1
        
        # Print e update das métricas
        # A condição *** and curr_iter < 1000 *** pode ser removida se você quiser treinar com o dataset completo
        if (i + 1) % 100 == 0 and curr_iter < 1000:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (epoch, 
                                                                                       i + 1, 
                                                                                       len(treino_loader), 
                                                                                       train_loss.avg, 
                                                                                       train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
            
    return train_loss.avg, train_acc.avg

In [12]:
total_loss_val, total_acc_val = [],[]
# Função para validação
def valida_modelo(val_loader, model, criterion, optimizer, epoch):
    
    # Coloca o modelo em modo de validação
    model.eval()
    
    # Inicializa objetos de cálculo de métricas
    val_loss = CalculaMetricas()
    val_acc = CalculaMetricas()
    
    # Validação
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            
            images, labels = data
            
            N = images.size(0)
            
            images = Variable(images).to(device)
            
            labels = Variable(labels).to(device)

            outputs = model(images)
            
            prediction = outputs.max(1, keepdim = True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    
    return val_loss.avg, val_acc.avg

### Editando dataset

In [13]:
y = labels['chord_idx']
_, df_validacao = train_test_split(labels, test_size = 0.2, random_state = 101, stratify = y)

In [14]:
df_validacao.shape

(7427, 7)

In [15]:
df_validacao['chord_idx'].value_counts()

chord_idx
186    45
20     44
303    43
109    42
334    42
       ..
92      6
197     6
181     6
227     6
120     6
Name: count, Length: 373, dtype: int64

In [16]:
# Esta função identifica se uma imagem faz parte do conjunto train ou val
def get_val_rows(x):
    val_list = list(df_validacao['clean'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

In [17]:
# Identifica treino ou validação
labels['train_or_val'] = labels['clean']
labels['train_or_val'] = labels['train_or_val'].apply(get_val_rows)

In [18]:
# Filtra as linhas de treino
df_treino = labels[labels['train_or_val'] == 'train']

In [19]:
print(len(df_treino))
print(len(df_validacao))

28783
7427


In [20]:
df_treino['chord_idx'].value_counts()

chord_idx
186    175
20     172
303    166
334    165
109    165
      ... 
169     26
227     25
181     25
120     24
197     22
Name: count, Length: 373, dtype: int64

In [21]:
df_validacao['chord'].value_counts()

chord
D#m11    45
A#m11    44
Fm11     43
C#m11    42
G#m11    42
         ..
C#5       6
D5        6
D#aug     6
E5        6
C5        6
Name: count, Length: 373, dtype: int64

In [ ]:
# Podemos dividir o conjunto de validação em um conjunto de validação e um conjunto de teste
df_validacao, df_teste = train_test_split(df_validacao, test_size = 0.5)

In [ ]:
# Reset do índice
df_validacao = df_validacao.reset_index()
df_teste = df_teste.reset_index()

In [ ]:
df_validacao.shape

In [ ]:
df_teste.shape

#### Inicializando modelo

In [ ]:
# Modelo que será treinado
# nome_modelo = 'densenet'
# nome_modelo = 'resnet'
# nome_modelo = 'inception'

In [ ]:
num_classes = 346

In [ ]:
# Vamos treinar o modelo e sempre atualizar os pesos
feature_extract = False

In [ ]:
# Inicializa o modelo
model_ft, input_size = inicializa_modelo(num_classes, feature_extract, use_pretrained = False)

In [ ]:
# Coloca o modelo no device
model = model_ft.to(device)

In [ ]:
import torchaudio.transforms as T

In [ ]:
target_sample_rate = 22050

In [ ]:
transform_treino = transforms.Compose([
    T.Resample(orig_freq=22050, new_freq=3500),  # sr é a taxa de amostragem original do áudio
    T.TimeStretch(),  # Para mudar a velocidade do áudio
    T.PitchShift(n_steps=30, n_fft=2048, sample_rate=target_sample_rate),  # Para mudar o pitch
    T.FrequencyMasking(freq_mask_param=15),  # Para adicionar máscaras de frequência
    T.TimeMasking(time_mask_param=35),  # Para adicionar máscaras de tempo
    # ... outras transformações específicas de áudio
    T.AmplitudeToDB(),
])

In [ ]:
# Transformações das imagens de validação
transform_val = transforms.Compose([
    #T.Resample(orig_freq=sr, new_freq=target_sample_rate),
    T.AmplitudeToDB(),
])

#### Carregando Dataloader

In [ ]:
# Organiza e transforma os dados de treino
set_treino = AudioDataset(df_treino, transform = transform_treino)
loader_treino = DataLoader(set_treino, batch_size = 32, shuffle = True, num_workers = 4)

In [ ]:
# O mesmo em validação
set_val = AudioDataset(df_validacao, transform = transform_val)
loader_val = DataLoader(set_val, batch_size = 32, shuffle = False, num_workers = 4)

In [ ]:
# O mesmo em teste
set_teste = AudioDataset(df_teste, transform = transform_val)
loader_teste = DataLoader(set_teste, batch_size = 32, shuffle = False, num_workers = 4)

In [ ]:
# Usaremos o otimizador Adam
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
# Usaremos cross entropy loss como função de perda
criterion = nn.CrossEntropyLoss().to(device)

#### Treinamento

In [ ]:
# Hiperparâmetros
epoch_num = 3
best_val_acc = 0

In [ ]:
%%time
for epoch in range(1, epoch_num + 1):
    
    # Execute a função de treino
    loss_train, acc_train = treina_modelo(loader_treino, model, criterion, optimizer, epoch)
    
    # Executa a função de validação
    loss_val, acc_val = valida_modelo(loader_val, model, criterion, optimizer, epoch)
    
    # Calcula as métricas
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    
    # Verifica a acurácia em validação
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        print('*****************************************************')
        print('Melhor Resultado: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')

        torch.save(model.state_dict(), f'resnet_model_{epoch}.pth')